In [90]:
from itertools import combinations
import numpy as np
import random
import matplotlib.pyplot as plt
import time as time

# Implementation

I solved the Traveling Salesman Problem (TSP) using a Genetic Algorithm (GA).
Each individual in the population is represented as a permutation of numbers corresponding to the graph’s nodes, all starting from node 0. In this way, every distinct permutation corresponds to a unique tour.

The initial population is generated from a set of different greedy solutions, where each solution is built by iteratively choosing the best next node available. I constructed these greedy paths starting from various nodes (including and excluding node 0), and then adjusted them so that each tour begins at node 0.
The cost (fitness) of an individual is defined as the total length of the path it represents.

The mutation operator consists of two types: inversion mutation and insertion mutation. Both are designed to avoid swapping or moving adjacent nodes, as this usually leads to minor changes with limited impact.

Parent selection is performed using a tournament selection strategy, where the tournament size increases progressively with each iteration (as a form of parameter control).
The mutation applied to an offspring is chosen randomly between inversion and insertion, according to a parameter called the inversion rate.
Recombination is performed using the Inver-Over crossover operator, which is always applied. Mutation, on the other hand, is applied a random number of times depending on two parameters: mutation_prob and mutation_rate. The mutation rate gradually decreases as generations progress.

The genetic algorithm terminates either after a fixed number of generations or if no improvement is observed for a specified number of consecutive generations.

To select appropriate parameters, I conducted several experiments with different configurations. The population size and the number of generations generally increase with the problem size. However, to ensure reasonable computation times for larger instances, I used smaller parameter values. Increasing these values would likely improve solution quality, although at the cost of higher computational effort and longer execution times.

## Functions

In [ ]:

def greedy_initial_solution(problem):
    n = problem.shape[0]
    unvisited = set(range(n))
    current_city = 0
    tour = [current_city]
    unvisited.remove(current_city)

    while unvisited:
        next_city = min(unvisited, key=lambda city: problem[current_city, city])
        tour.append(next_city)
        unvisited.remove(next_city)
        current_city = next_city

    return tour

In [ ]:
#this is a function to create a number of greedy solutions starting from a random city but having city 0 at the start
def greedy_initial_solution_randomstart(problem):
    """Greedy initial solution starting from a random city between 1 and n-1. At the end i put city 0 at the front to have a complete tour starting from 0
.
    """
    n = problem.shape[0]
    if n == 1:
        return [0]

    # unvisited considers only elements different from 0
    unvisited = set(range(1, n))

    import random
    # pick an integer random tra 1 e n-1 (inclusi)
    current_city = random.randint(1, n - 1)
    tour = [current_city]
    unvisited.remove(current_city)

    while unvisited:
        next_city = min(unvisited, key=lambda city: problem[current_city, city])
        tour.append(next_city)
        unvisited.remove(next_city)
        current_city = next_city

    # put city 0 at the front so the tour starts with 0
    if 0 not in tour:
        tour = [0] + tour

    return tour


In [ ]:
#random element to fill the population with random permutations starting with 0
def permutation_starting_zero(n):
    """Return a random permutation of 0..n-1 that starts with 0."""


    # shuffle the remaining elements (1..n-1) and prepend 0
    rest = list(range(1, n))
    import random
    random.shuffle(rest)
    return [0] + rest


In [ ]:
def BuildInitialPopulation(problem, population_size, greedy_population):
    """Build an initial population of given size using greedy_initial_solution_randomstart.
    """
    population = []
    individual = greedy_initial_solution(problem)
    population.append(individual)
    for _ in range(greedy_population):
        individual = greedy_initial_solution_randomstart(problem)
        population.append(individual)

    for _ in range(population_size-greedy_population-1):
        individual = permutation_starting_zero(problem.shape[0])
        population.append(individual)
    return population

In [ ]:
#cost function
def calculate_tour_length(problem, tour):
    length = 0
    n = len(tour)
    for i in range(n):
        length += problem[tour[i], tour[(i + 1) % n]]
    return length

Mutation and recombination function

In [ ]:

def mutate_inversion(tour):
    """Mutate the tour by inverting a random segment (excluding the first city).
    """
    n = len(tour)
    if n <= 3:
        return tour  # No mutation possible

    # Select two random indices for the segment to invert, ensuring they are not the first city
    i, j = sorted(random.sample(range(1, n), 2))

    # Invert the segment between indices i and j
    tour[i:j + 1] = reversed(tour[i:j + 1])

    return tour

In [17]:
def mutate_insertion(tour):
    """Mutate the tour by removing a city and inserting it at a different position (excluding the first city).
    """
    n = len(tour)
    if n <= 3:
        return tour  # No mutation possible

    # Select a random index to remove, ensuring it's not the first city
    remove_index = random.randint(1, n - 1)
    city = tour.pop(remove_index)

    # Select a new position to insert the city, ensuring it's not the first position
    insert_index = random.randint(1, n - 2)  # n-2 because we just removed one city

    # Insert the city at the new position
    tour.insert(insert_index, city)

    return tour

In [18]:
def crossover_inver_over(parent1, parent2):
    """Perform inversion crossover between two parent tours.
    """
    n = len(parent1)
    if n <= 3:
        return parent1.copy(), parent2.copy()  # No crossover possible
    flag=True
    offspring=parent1.copy()
    while flag:

            # Select two random indices for the segment to invert
            i = random.choice(range(1, n))

            # Create offspring by inverting the segment between indices i and j
            city=parent1[i]
            j=parent2.index(city)
            city2=parent2[j+1 if j+1<n else 1]
            if city2 in parent1[i:]:
                k=parent1.index(city2)
                offspring[i+2:k+1 ] = parent1[i+1:k]
                offspring[i+1] = city2
                flag=False 
 


    return offspring

Evaluation and parent selection functions

In [19]:
def evaluate_population(problem, population):
    """Evaluate the population and return a list of (tour, length) tuples.
    """
    evaluated = []
    for individual in population:
        length = calculate_tour_length(problem, individual)
        evaluated.append((individual, length))
    return evaluated

In [20]:
def parent_selection_tournament(evaluated_population, tournament_size):
    """Select a parent using tournament selection.
    """
    tournament = random.sample(evaluated_population, tournament_size)
    tournament.sort(key=lambda x: x[1])  # Sort by length (fitness)
    return tournament[0][0]  # Return the best individual's tour

Genetic algorithm function

In [82]:
def genetic_algorithm(problem, population_size=100, offspring_size=50, generations=500, mutation_rate=0.8, mutation_prob=0.3, invert_prob=0.4, greedy_population=10, max_tournament_size=5, no_better=50, tournament_rate=1.05, weight=1.5, weight_rate=1.02):
    # Build initial population
    population = BuildInitialPopulation(problem, population_size, greedy_population)
    tournament_size=max_tournament_size
    tournament_upgrade=tournament_size
    best_element=None
    

    evaluated_population = evaluate_population(problem, population)
    evaluated_population.sort(key=lambda x: x[1])  # Sort by tour length

    for generation in range(generations):
        next_generation = population.copy()
        tournament_upgrade=tournament_upgrade*tournament_rate
        tournament_size= min(int(tournament_upgrade), population_size)
        mutation_rate=mutation_rate-0.001
        # weight=weight*weight_rate
        # int_weight=int(weight)

        for x in range(offspring_size):


            parent1, parent2 = parent_selection_tournament(evaluated_population, tournament_size), parent_selection_tournament(evaluated_population, tournament_size)
            #parent1, parent2 = parent_selection_ranked(evaluated_population, int_weight), parent_selection_ranked(evaluated_population, int_weight)
            offspring = crossover_inver_over(parent1, parent2)
            if random.random() < mutation_prob:
                while random.random() < mutation_rate:
                    
                    if random.random() < invert_prob:
                        offspring = mutate_insertion(offspring)
                    else:
                        offspring = mutate_inversion(offspring)
        
            next_generation.append(offspring)

        evaluated_population = evaluate_population(problem, next_generation)
        evaluated_population.sort(key=lambda x: x[1])  # Sort by tour length
        population = [ind for ind, length in evaluated_population[:population_size]]
        if best_element is None or evaluated_population[0][1] < best_element[1]:
            best_element = evaluated_population[0]
            upgradecounter=0
        else:   
            upgradecounter+=1 
        if upgradecounter>no_better:
            print(f"Stopping early at generation {generation} with best length {best_element[1]}")
            break 


    best_tour, best_length = evaluated_population[0]
    return best_length

In [ ]:
def tuning_parameters(problem_list, generations=500, population_size=500, offspring_size=250, greedy_population=80):
    """Tune parameters on a list of problems and return the best configuration and plot data.

    Returns:
        best_config: tuple (mutation_prob, mutation_rate, invert_prob)
        plot_results: list of tuples (mutation_prob, mutation_rate, invert_prob, avg_length)
    """
    best_config = None
    best_average_length = float('inf')
    plot_results = []

    # Example parameter ranges to tune (kept moderate to avoid extremely long runs)
    mutation_probs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    mutation_rates = [0.1, 0.2, 0.3, 0.4, 0.5]
    invert_probs = [0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9]

    runs_per_problem = 5

    for mp in mutation_probs:
        for mr in mutation_rates:
            for ip in invert_probs:
                total_length = 0.0
                for problem in problem_list:
                    for i in range(runs_per_problem):  # Run multiple times for averaging
                        length = genetic_algorithm(problem, generations, population_size, offspring_size, mr, mp, ip, greedy_population, no_better=100)
                        total_length += length
                average_length = total_length / (len(problem_list) * runs_per_problem)
                plot_results.append((mp, mr, ip, average_length))

                if average_length < best_average_length:
                    best_average_length = average_length
                    best_config = (mp, mr, ip)


    print("Mutation probability, Mutation rate, Inversion Probability")

    return best_config, plot_results

# implementation

We have used small value for population_size and generation to make it faster. The results find are not the best findable but are done in a fast time.

In [ ]:
cont=0
pop_size=25
generation=25
best=0.0
problems=[np.load('lab2/problem_g_10.npy'), np.load('lab2/problem_g_20.npy'), np.load('lab2/problem_g_50.npy'), np.load('lab2/problem_g_100.npy'), np.load('lab2/problem_g_200.npy'), np.load('lab2/problem_g_500.npy'), np.load('lab2/problem_g_1000.npy')]
for prob in problems:
    pop_size+=int(pop_size/3*2)
    generation+=int(pop_size/7*5)
    if prob.shape[0]==1000:
        pop_size=500
        generation=800

    time_start=time.time()
    best=genetic_algorithm(prob, population_size=pop_size, greedy_population=int(pop_size*0.3), offspring_size=int(pop_size*0.5), generations=generation, mutation_rate=0.5, mutation_prob=0.9, invert_prob=0.5, no_better=int(generation*0.25))
    time_end=time.time()
    print(f"Problem size: {prob.shape[0]}, Best length: {best}, time: {time_end-time_start}" )

Stopping early at generation 21 with best length 1497.663648225291
Problem size: 10, Best length: 1497.663648225291, time: 0.024847030639648438
Stopping early at generation 64 with best length 1815.1711244105356
Problem size: 20, Best length: 1815.1711244105356, time: 0.09404325485229492
Stopping early at generation 105 with best length 2860.0966004337247
Problem size: 50, Best length: 2860.0966004337247, time: 0.6099863052368164
Stopping early at generation 105 with best length 2860.0966004337247
Problem size: 50, Best length: 2860.0966004337247, time: 0.6099863052368164
Stopping early at generation 168 with best length 4598.329499022724
Problem size: 100, Best length: 4598.329499022724, time: 4.55049204826355
Stopping early at generation 168 with best length 4598.329499022724
Problem size: 100, Best length: 4598.329499022724, time: 4.55049204826355
Problem size: 200, Best length: 6040.236846118214, time: 32.36966896057129
Problem size: 200, Best length: 6040.236846118214, time: 32.36

In [128]:
cont=0
pop_size=30
generation=30
best=0.0
problems=[np.load('lab2/problem_r1_10.npy'), np.load('lab2/problem_r1_20.npy'), np.load('lab2/problem_r1_50.npy'), np.load('lab2/problem_r1_100.npy'), np.load('lab2/problem_r1_200.npy'), np.load('lab2/problem_r1_500.npy'), np.load('lab2/problem_r1_1000.npy')]
for prob in problems:
    pop_size+=int(pop_size/3*2) 
    generation+=int(pop_size/7*5)
    if prob.shape[0]==1000 or prob.shape[0]==500:
        pop_size=400
        generation=600

    time_start=time.time()
    best=genetic_algorithm(prob, population_size=pop_size, greedy_population=int(pop_size*0.3), offspring_size=int(pop_size*0.9), generations=generation, mutation_rate=0.9, mutation_prob=0.9, invert_prob=0.9, no_better=int(generation*0.25))
    time_end=time.time()
    print(f"Problem size: {prob.shape[0]}, Best length: {best}, time: {time_end-time_start}" )

Stopping early at generation 31 with best length 188.9273581837241
Problem size: 10, Best length: 188.9273581837241, time: 0.05551648139953613
Stopping early at generation 67 with best length 343.6202496272525
Problem size: 20, Best length: 343.6202496272525, time: 0.4744744300842285
Stopping early at generation 67 with best length 343.6202496272525
Problem size: 20, Best length: 343.6202496272525, time: 0.4744744300842285
Stopping early at generation 56 with best length 630.2951293403518
Problem size: 50, Best length: 630.2951293403518, time: 1.05226469039917
Stopping early at generation 56 with best length 630.2951293403518
Problem size: 50, Best length: 630.2951293403518, time: 1.05226469039917
Stopping early at generation 239 with best length 774.3074335864989
Problem size: 100, Best length: 774.3074335864989, time: 16.856016874313354
Stopping early at generation 239 with best length 774.3074335864989
Problem size: 100, Best length: 774.3074335864989, time: 16.856016874313354
Probl

In [127]:
cont=0
pop_size=30
generation=30
best=0.0
problems=[np.load('lab2/problem_r2_10.npy'), np.load('lab2/problem_r2_20.npy'),np.load('lab2/problem_r2_50.npy'), np.load('lab2/problem_r2_100.npy'), np.load('lab2/problem_r2_200.npy'), np.load('lab2/problem_r2_500.npy'), np.load('lab2/problem_r2_1000.npy')] 
for prob in problems:
    pop_size+=int(pop_size/3*2) 
    generation+=int(pop_size/7*5)
    if prob.shape[0]==1000 or prob.shape[0]==500:
        pop_size=400
        generation=700

    time_start=time.time()
    best=genetic_algorithm(prob, population_size=pop_size, greedy_population=int(pop_size*0.3), offspring_size=int(pop_size*0.9), generations=generation, mutation_rate=0.9, mutation_prob=0.7, invert_prob=0.9, no_better=int(generation*0.4))
    time_end=time.time()
    print(f"Problem size: {prob.shape[0]}, Best length: {best}, time: {time_end-time_start}" )

Stopping early at generation 27 with best length -411.7017155524984
Problem size: 10, Best length: -411.7017155524984, time: 0.036664724349975586
Stopping early at generation 107 with best length -752.4537280674261
Problem size: 20, Best length: -752.4537280674261, time: 0.6146993637084961
Stopping early at generation 107 with best length -752.4537280674261
Problem size: 20, Best length: -752.4537280674261, time: 0.6146993637084961
Stopping early at generation 151 with best length -2209.2103654187536
Problem size: 50, Best length: -2209.2103654187536, time: 2.901966094970703
Stopping early at generation 151 with best length -2209.2103654187536
Problem size: 50, Best length: -2209.2103654187536, time: 2.901966094970703
Stopping early at generation 155 with best length -4622.157405816591
Problem size: 100, Best length: -4622.157405816591, time: 10.189958810806274
Stopping early at generation 155 with best length -4622.157405816591
Problem size: 100, Best length: -4622.157405816591, time: